# Sample PySpark Notebook

See the README for more details a link to the home repo

In [1]:
import pyspark
sc = pyspark.SparkContext('local[*]')

In [2]:
# do something to prove it works
rdd = sc.parallelize(range(1000))
rdd.takeSample(False, 5)

[797, 656, 448, 139, 318]

In [4]:
# Quick example of word counts by line

# lines = sc.textFile("file:///tmp/sampleFile.txt")
lines = sc.parallelize([
    "This is line one",
    "This is line two",
    "Last line."
])

def words(line_index):
    line, index = line_index
    return (index, line.split(" "))

lines.zipWithIndex().map(words).map(
    lambda kv: (kv[0], len(kv[1]))
).reduceByKey(
    lambda x,y: x+y
).collect()

[(0, 4), (1, 4), (2, 2)]